# <center><font color="green"> https://bit.ly/ptpjb-2021-09</font></center>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://bit.ly/ptpjb-2021-09)

# <center><font color="blue">09 - Time Series Analysis - (S)Arima</font></center>

<center><img alt="" src="images/cover_ptpjb_2021.png"/></center> 

## <center><font color="blue">tau-data Indonesia</font><br>(C) Taufik Sutanto - 2021</center>
<center><a href="https://tau-data.id">https://tau-data.id</a> ~ <a href="mailto:taufik@tau-data.id">taufik@tau-data.id</a></center>

# <center><font color="blue"> 09 - Time Series Analysis - (S)Arima</font></center>

* Pendahuluan Time Series Analysis
* Stasioneritas & Autokorelasi Arima
* Parameter & Spesifikasi Model Arima
* Sarima - Seasonality
* Studi kasus Arima & Sarima

<img src="images/time series.png" style="height: 200px;"/>

image source: https://medium.com/@paradistia/analisis-runtun-waktu-peramalan-fuzzy-time-series-cheng-menggunakan-microsoft-excel-b33a192013bb

# <center><font color="blue">Data Bergantung Waktu</font></center>

* Beberapa data tertentu bergantung terhadap waktu, sebut saja pergerakan nilai mata uang (kurs)/harga saham, suhu/temperature udara di suatu daerah tertentu, atau data logs suatu website. 
* Saat nilai data di masa depan lebih banyak (dominan) hanya dipengaruhi dari nilai-nilainya di masa lalu, maka model-model runtun waktu univariate(satu peubah/variabel) seperti ARIMA (Autoregressive Integrated Moving Average) dapat digunakan. 
* Namun bila satu atau beberapa peubah yang bergantung waktu dipengaruhi juga oleh variable lain selain nilai-nilainya di masa lalu, maka model runtun waktu peubah ganda (multivariate) seperti VaR (Vector autoRegression) dapat digunakan. 

<img src="images/Time_Series_Data.png" style="height: 300px;"/>

# <center><font color="blue">Komponen Data Time Series</font></center>

Pola dari data time series dapat dibagi menjadi komponen-komponen berikut:
1. **Base atau Level**. Data time series yang diobservasi.
2. **Trend**. Kemiringan meningkat atau menurun yang terlihat di Time Series.
3. **Seasonality**. Pola unik yang terlihat di suatu interval waktu karena faktor musiman. Hal ini bisa karena suatu bulan di setiap tahunnya, suatu hari di setiap bulannya, atau bahkan suatu jam di dalam satu hari.
4. **Residual atau Noise atau Error**. Variasi dari data yang tidak dapat dijelaskan.

* Sebagai catatan, tidak semua data Time Series memiliki seluruh komponen diatas. Time Series akan selalu memiliki Base, rata-rata memiliki Residual, dan Trend dan Seasonality biasanya tidak harus ada.

<img src="images/time series decomposition trend seasonal.jpeg" style="height: 350px;"/>

# <center><font color="blue">ARIMA (Autoregressive Integrated Moving Average)</font></center>

Model ARIMA merupakan model statistik untuk menganalisis dan meramalkan data time series.

ARIMA adalah singkatan dari *AutoRegressive Integrated Moving Average* yang merupakan gabungan dari AutoRegressive dan Moving Average yang lebih sederhana dan menambahkan Integrated.
- **AR** : Autoregression. Adalah model yang menggunakan hubungan ketergantungan antara data saat ini dengan beberapa data di masa lalu.
- **I** : Integrated. Adalah penggunaan diferensiasi pada data observasi untuk membuatnya menjadi stasioner.
- **MA** : Moving Average. Adalah model yang menggunakan hubungan ketergantungan antara data saat ini dengan error dari hasil forecasting di masa lalu

<img src="images/arima_flowchart.jpg" style="height: 250px;"/>

# <center><font color="blue">Stasionaritas</font></center>

Data time series yang stasioner adalah data yang sifatnya tidak bergantung pada waktu. Oleh karena itu, deret waktu dengan tren, atau dengan musim, tidak stasioner. Metode ARIMA mengasumsikan bahwa data time series bersifat stasioner.

<img src="images/arima-time-series-stationarity.png" style="height: 350px;"/>

# <center><font color="blue">Contoh Kasus yang Tidak Stasioner</font></center>

* Y_t=GDP saat waktu t.
* y_t=log(Y_t) bukan proses yang stasioner
* y_t−y_(t−1) biasanya sudah merupakan proses yang stasioner

* S_t=harga suatu portofolio.
* s_t=log(S_t) biasanya bukan proses yang stasioner.
* Log return, r_t=s_t−s_(t−1) biasanya sudah merupakan proses yang stasioner.

# <center><font color="blue">Parameter p, d, q di ARIMA</font></center>

Masing-masing komponen ini secara eksplisit ditentukan dalam model sebagai parameter **p, d, dan q**. biasanya model arima dinotasikan **ARIMA(p,d,q)**. masing-masing paramater tersebut didefinisikan sebagai berikut:
- p: orde untuk proses autoregressive (AR).
- d: orde yang menyatakan banyaknya proses diferensiasi yang dilakukan pada data time series yang tidak stasioner.
- q: orde yang menyatakan proses moving average (MA).

<img src="images/Arima_Model.png" style="height: 250px;"/>

# <center><font color="blue">Spesifikasi Model: ACF - PACF</font></center>

<img src="images/time series formula acf pacf.png" style="height: 200px;"/>
<img src="images/acf-pacf-arima-graph.png" style="height: 250px;"/>

# <center><font color="blue">Parameter p & q</font></center>

* Menentukan p dan q berdasarkan pola ACF dan PACF

<img src="images/pola ACF PACF.jpg"  style="height: 300px;"/>

# <center><font color="blue">Pemilihan Model Berdasarkan Pola ACF dan PACF</font></center>

<img src="images/pola-acf-pacf.png" style="height: 250px;"/>

# <center><font color="blue">Pemilihan Model Berdasarkan Pola ACF dan PACF</font></center>

<img src="images/Diagnostic_model_arima.png" style="height: 400px;"/>

# <center><font color="blue">Variasi Arima</font></center>

ARIMA memiliki beberapa variasi di antaranya adalah:

- **ARIMA**: ARIMA dasar yang tidak dapat digunakan pada data musiman
- **Seasonal ARIMA (SARIMA)**: digunakan untuk meramalkan data berpola musiman
- **Seasonal ARIMA with eXogenous (SARIMAX)**:  digunakan untuk meramalkan data berpola musiman dengan beberapa variabel independen yang mempengaruhinya.

<img src="images/sarima.png" style="height: 100px;"/><img src="images/sarima_equation.png" style="height: 200px;"/>

# <center><font color="blue">Time Series Analysis in Python</font></center>

* Contoh kasus **Arima**: Data Penumpang Pesawat https://www.kaggle.com/rakannimer/air-passengers
* Contoh kasus **Sarima**: Produksi Energy Listrik & gas  https://fred.stlouisfed.org/series/IPG2211A2N

<img src="images/arima in python.jpg" style="height: 200px;"/>

In [ ]:
!pip install --upgrade pmdarima chart_studio plotly statsmodels cufflinks
# menginstall module yang dibutuhkan

In [ ]:
import warnings; warnings.filterwarnings("ignore")
import numpy as np, pandas as pd, pmdarima
import matplotlib.pyplot as plt, seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.graph_objects as go
import cufflinks as cf; cf.go_offline(); init_notebook_mode(connected=True)
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima.arima.utils import ndiffs
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

sns.set(style="ticks", color_codes=True)
random_state = 99
"Done"

# <center><font color="blue"> Contoh Kasus Data Penumpang Pesawat </font></center>

* Permasalahan: Prediksi jumlah penumpang maskapai pada suatu waktu tertentu.

  Data terdiri dari pengamatan selama 144 bulan, diukur sejak Januari 1949 hingga Desember 1960 (12 tahun).

* Sumber Data https://www.kaggle.com/rakannimer/air-passengers
* Kernel: https://towardsdatascience.com/machine-learning-part-19-time-series-and-autoregressive-integrated-moving-average-model-arima-c1005347b0d7

<img src="images/ilustrasi_arima_passenger_prediction.jpg" style="height: 250px;"/>

In [ ]:
# Loading Data menggunakan waktu sebagai index sebagaimana yang sudah di contohkan di module sebelumnya.

file_ = "data/AirPassengers_09.csv"
try: # Running Locally, yakinkan "file_" berada di folder "data"
    df = pd.read_csv(file_, parse_dates=True, index_col = "Month", error_bad_lines=False, low_memory = False, encoding='utf8')
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/ptpjb/master/{file_}
    df = pd.read_csv(file_, parse_dates=True, index_col = "Month", error_bad_lines=False, low_memory = False, encoding='utf8')

N, P = df.shape # Ukuran Data
print('baris = ', N, ', Kolom (jumlah variabel) = ', P)
print(df.info())
df.head()

In [ ]:
df.describe(include="all"), min(df.index), max(df.index)

# <center><font color="blue"> Visualisasi Data </font></center>

In [ ]:
p = df.iplot(title="Penumpang Pesawat 1949~1960")
# Nampak jelas adanya Trend

# <center><font color="blue">Stationarity</font></center>

* Pastikan bahwa runtun waktu bersifat stasioner sebelum melakukan pemodelan. Dua cara pemeriksaan asumsi stasioner adalah:


   * **Rolling Statistics**: Buat grafik rolling mean dan standar deviasi. Runtun waktu dikatakan stasioner jika mean dan dan standar deviasi konstan terhadap waktu. Sudah dibahas di Module 04 (Visualisasi).

   * **Augmented Dickey-Fuller Test**:  Runtun waktu dikatakan stasioner jika p-value kecil dan nilai kritis pada taraf signifikansi 1%, 5%, 10% dekat dengan nilai statistik ADF.

In [ ]:
rolling_mean = df.rolling(window = 12).mean()
rolling_std = df.rolling(window = 12).std()
plt.plot(df, color = 'blue', label = 'Original')
plt.plot(rolling_mean, color = 'red', label = 'Rolling Mean')
plt.plot(rolling_std, color = 'black', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Rolling Standard Deviation')
plt.show()

# <center><font color="blue">Rolling Mean & Standard Deviation</font></center>

* Visual: Rolling mean dan rolling standard deviation meningkat seiring berjalannya waktu--> runtun tidak stasioner
* Uji statistik formal

In [ ]:
result = adfuller(df['Passengers'])
print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

# <center><font color="blue">ADF Statistic</font></center>

* Statistik ADF berbeda jauh dengan nilai kritis; p-value > threshold (0.05) --> runtun waktu tidak stasioner.

* Gunakan transformasi log untuk mengurangi tingkat ketidakstasioneran.


In [ ]:
df_log = np.log(df)
plt.plot(df_log)

# <center><font color="blue">Transformasi & Koreksi mean</font></center>

* Lakukan transformasi untuk mendapatkan runtun yang stasioner.
* misal dengan koreksi mean.

In [ ]:
def get_stationarity(timeseries, judul, tipe=1, dt=None):
    if tipe==1:
        original = plt.plot(timeseries, color='blue', label='Original')
        rolling_mean = timeseries.rolling(window=12).mean()
        rolling_std = timeseries.rolling(window=12).std()
        mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
        std = plt.plot(rolling_std, color='black', label='Rolling Std')
    else: #tipe !=1 & dt != None
        original = plt.plot(timeseries, color='blue', label='Difference')
        diff = plt.plot(dt, color='black', label='Original')
    plt.legend(loc='best')
    plt.title(judul)
    plt.show(block=False)
    result = adfuller(timeseries['Passengers'])
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

In [ ]:
df_log = np.log(df)
rolling_mean = df_log.rolling(window=12).mean()
df_log_minus_mean = df_log - rolling_mean
df_log_minus_mean.dropna(inplace=True)
get_stationarity(df_log_minus_mean, "Rolling Mean & Standar Deviation")

# <center><font color="blue"> Differencing </font></center>

* The ARIMA (aka Box-Jenkins) model adalah "perluasan" dari ARMA model, data runtun waktu di-differencing terlebih dahulu untuk mendapatkan runtun yang stasioner, kemudian dimodelkan dengan ARMA.  
* Contoh: 

 * First-order differencing, $z_i = y_i-y_{i-1}$. 
 * Second-order differencing, $z_i = (y_i-y_{i-1})-(y_{i-1}-y_{i-2})$, dan seterusnya.

<img src="images/arima-differencing.png" style="height: 250px;"/>

In [ ]:
df_log_shift = df_log - df_log.shift()
df_log_shift.dropna(inplace=True)
get_stationarity(df_log_shift, "Differencing", tipe=2, dt=df_log)

# <center><font color="blue">Hasil Pengamatan</font></center>

* Runtun sudah stasioner; dapat dilakukan pemodelan.
* Periksa ACF dan PACF terlebih dahulu.

In [ ]:
# Jika data tidak statsioner lakukan diferensiasi (y_t - y_t-1)
#diff = df['Passengers'].diff().dropna()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))
plot_pacf(df_log_shift, ax=ax1)
plot_acf(df_log_shift, ax=ax2)
plt.show()

# <center><font color="blue">Dapat mengajukan beberapa pilihan model</font></center>

* ARIMA(2,1,2)
* ARIMA(3,1,2)
* ARIMA(3,1,4)
* ARIMA(4,1,4)

Pertimbangkan trade-off antara akurasi dan kompleksitas model.

In [ ]:
decomposition = seasonal_decompose(df_log) 
model = ARIMA(df_log, order=(3,1,2))
results = model.fit(disp=-1)
plt.plot(df_log_shift)
plt.plot(results.fittedvalues, color='red')
plt.show()

# <center><font color="blue">Bandingkan hasil peramalan model dengan data sesungguhnya</font></center>

In [ ]:
predictions_ARIMA_diff = pd.Series(results.fittedvalues, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(df_log['Passengers'].iloc[0], index=df_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(df)
plt.plot(predictions_ARIMA)
plt.show()

In [ ]:
"""
Forecast jumlah penumpang untuk 10 tahun (10 x 12 bulan) ke depan, 
ukuran data (12x12) + (12 x 10) = 264
"""
results.plot_predict(1,264)
plt.show()

In [ ]:
results.summary()

In [ ]:
stepwise_model = auto_arima(df, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=False,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

# <center><font color="blue"> Contoh Kasus SARIMA: Data NAICS </font></center>

* Permasalahan: Data bulanan produksi **Listrik dan Gas** sebuah perusahaan di Amerika Serikat dari tahun 1985-2018.
* Sumber Data https://fred.stlouisfed.org/series/IPG2211A2N
* Contoh kasus menggunakan **Auto Arima** untuk memudahkan pemodelan time Series di Python.

<img src="images/data-naics-time-series-energy.png" style="height: 250px;"/>

In [ ]:
# Loading Data menggunakan waktu sebagai index sebagaimana yang sudah di contohkan di module sebelumnya.

file_ = "data/IPG2211A2N.csv"
try: # Running Locally, yakinkan "file_" berada di folder "data"
    df = pd.read_csv(file_, parse_dates=True, index_col = "DATE", error_bad_lines=False, low_memory = False, encoding='utf8')
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/ptpjb/master/{file_}
    df = pd.read_csv(file_, parse_dates=True, index_col = "DATE", error_bad_lines=False, low_memory = False, encoding='utf8')

df.columns = ['Energy-Production'] # Rename Kolom
N, P = df.shape # Ukuran Data
print('baris = ', N, ', Kolom (jumlah variabel) = ', P)
print(df.info())
df.head()

# <center><font color="blue"> Visualisasi Data </font></center>

In [ ]:
p = df.iplot(title="Energy Production Jan 1985--Jan 2018")

# <center><font color="blue"> Dekomposisi Karena Adanya Trend </font></center>

* Terlihat nampak ada Trend naik.
* Dekomposisi runtun waktu adalah sebuah proses statistik untuk mendekomposisi (deconstruct) sebuah data runtun  


In [ ]:
result = seasonal_decompose(df, model="multiplicative")
fig = result.plot()

# <center><font color="blue">Trend and seasonality</font></center>

* Ada trend naik.
* Ada seasonal
* Ada irregular komponen

# <center><font color="blue">Performing the Seasonal ARIMA</font></center>

* Karena ada seasoal trend, ajukan model SARIMA, dengan (p,d,q) untuk komponen ARIMA, dan (P,D,Q) untuk komponen seasonal.
* Penentuan p,d,q,P,D,Q berdasarkan ACF dan PACF; dapat diajukan beberapa model.
* Penentuan model terbaik berdasarkan AIC.


In [ ]:
stepwise_model = auto_arima(df, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

# <center><font color="blue">Train Test Split</font></center>

* Dilakukan untuk menilai kelayakan model.
* Train data: sebagian (besar) data awal; pada contoh data tahun 1985-2015
* Test data: sebagian (kecil) data akhir.



In [ ]:
train = df.loc['1985-01-01':'2016-12-01']
test = df.loc['2017-01-01':]

# <center><font color="blue">Train the Model</font></center>

* Train model dengan menggunakan .fit pada model stepwise dan menerapkannya pada data training.

In [ ]:
stepwise_model.fit(train)

In [ ]:
stepwise_model.summary()

# <center><font color="blue">Evaluasi</font></center>

* Lakukan forecast pada data test, periode 1 Januari 2015 hingga1 Januari 2018.

In [ ]:
future_forecast = stepwise_model.predict(n_periods=55)
# This returns an array of predictions:
print(future_forecast)

# <center><font color="blue">Visualisasikan Prediksi</font></center>

In [ ]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['Prediction'])
pd.concat([test,future_forecast],axis=1).iplot()

# <center><font color="blue">Membandingkan dengan Seluruh Data</font></center>

In [ ]:
pd.concat([df,future_forecast],axis=1).iplot()

# <center><font color="blue"> Akhir Modul 09 - Time Series Analysis - (S)Arima</font></center>

<hr />
<img alt="" src="images/meme-cartoon/TimeSeries_MeMe.jpg" style="height: 300px;"/>